In [2]:
import pandas as pd
import numpy as np
from summarytools import dfSummary
import plotly.express as px 

import warnings 
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv("wine_quality_classification.csv")
data.head()

,fixed_acidity,residual_sugar,alcohol,density,quality_label
0,9.3,6.4,13.6,1.0005,high
1,11.2,2.0,14.0,0.9912,medium
2,11.6,0.9,8.2,0.9935,low
3,12.9,6.6,12.7,1.0002,low
4,13.9,13.8,10.4,0.9942,medium


## 2. Data Quality Checks

In [5]:
dfSummary(data, is_collapsible=False)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,fixed_acidity[float64],Mean (sd) : 10.2 (3.5)min < med < max:4.0 < 10.4 < 16.0IQR (CV) : 6.0 (2.9),121 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACgElEQVR4nO3dPY7aUBSG4XvDgAxIttBI0LGBSCkoWUQWmyWkYAXsgA4JISM8Y2GMUyVSioB/uOF88D69mVu8g8890mh8VVUOsO7Low8A1PH26APgb977oXNu0OEjTlVVfd7rPFYQqiHe++H7+/v3OI4nbT/jcDjsvfc/ni1WQrVlEMfxZLlcfiZJkjd9OE3TaLVaTXa73cA5R6gIK0mSfDqdfrR8fHjXwxjBZQoS+EbFH5YvcoQK55z9ixyh4jfTFzlCDaDDKzS+XC79e5+nCasXOUK9sy6v0KIoorIsv+Z5/tM51zaWhynLsu+ci733bR6/Ot8S6v21foVuNpvJer0enc/nXqjDhZJlWb8oim/z+bzX6/Uajw635ltCDaTNK3S/38vuQE+n01sURaPFYpHPZrN9k2frzLeE+g/Kc+YjjcfjtjPu1V/SoKFa3std88pzplXBQrW+l7vhJedMy0J+o5rey9XxanOmZcFnVKt7OWgxfZkKuZeDFrOhht7LQYvZUEPv5aDlZqiP3ie23cuVZRm79mPDS+9CLboaquo+sevYwC7UnlvfqJL7xC5jg3PsQi2qNaOq7hPbjg0Wzt5Wh02J6XHH7GUKzXUZeayPO4T6RLqMPNbHHUJ9Qm1GHuvjDn8uDQmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgmECgm1/iFamqZR0w8+Ho+Rc85lWRZtt9vR/3xe9Wernrvr83X6+gXdMrLf3JwiugAAAABJRU5ErkJggg=="">",0(0.0%)
2,residual_sugar[float64],Mean (sd) : 7.7 (4.2)min < med < max:0.5 < 7.5 < 15.0IQR (CV) : 7.4 (1.8),145 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACjElEQVR4nO3dTY7aQBRF4VehQQYkW4gIZmwgUgYMWUQWmyUEiRWwA2ZICBnhtMVvRpl1Guyi4rpwvnk1Hpxnm9cttbterwbE7kvTFwDc463pC4iVc65rZp2axw/X6/X9kdfz6gj1A8657nA4/JGm6aDO+d1ut3XO/VSL1XM4zQIOaNShNnhX66RpOpjNZu9ZlpVVDuZ5niwWi8Fms+mYmUyovsNpFnZAow01hrtalmXlaDT6XeNot+5nNqj2cJqFH9BoQ7UXvKvFwGM4zQIOaMyhmtnL3dXwD6ynIIFQIYFQIeHmOyqLb8Tg01BjWBEpOp/PbTNLnXN1jjPcH7h1R2VFVFFRFO3j8fh9Mpm0Wq1W5X2k8nCHHNC71lOsiO53OBzekiTpTafTcjweb6ucVR7u0AMa/R5VVb/ff6nhDj2ghIqHCjWgrKcggVAhgVAhgVAhIeiXKc+9Wnq5XNoPviSIChaq717teDwm5/P5W1mWv8ys7t9H4kkEC9Vnr2ZmtlqtBsvlsnc6nVohri9Wnk8hsyf9FWzwPWrdvdp2u/VafCu+dvg+hcy0fwX7madc+Ku+dvg+hfI8T+bz+Wiz2Xx1zu0qHo/6O8FThqr+2lH3KeQzoLF/J3jKUP9q6rWjKT4D2vRw3vLUob6qOgMa+3Cy8IcEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoUEQoWEu/4hWp7nSdUfvN/vEzOzoiiS9Xrd+5/nVT9b9bp9z9/T1x+nO8ODKf5h4AAAAABJRU5ErkJggg=="">",0(0.0%)
3,alcohol[float64],Mean (sd) : 11.0 (1.8)min < med < max:8.0 < 11.0 < 14.0IQR (CV) : 3.1 (6.3),61 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACvElEQVR4nO3du47aQBSA4TNhQQNItrgIOl5gpRRINDxAyjxsHiFIFFvzBnRICIFw1sIYp0ya5eLxYB/zf+1q1l7rlzVzXKzJskyAqvtW9g0A93gr+wZQH8aYtoi0ci4/ZVn2+dUPCRWFMMa0B4PBzyAIennWHw6HnTHm11exEiqK0gqCoDefzz/DMIwfWbjf7+1yuextt9uWiBAq/AvDMB6NRn9yLG1f+yGHKahAqFCBUKFCbfeojqMSkRvjkjpyfGbB5XJpFnk//6tlqK6jEpHb45K6cX1mSZLYNE3f4zj+LSJ5DlNX1TJUcRiViNw3Lqkhp2e2Xq97q9Wqcz6fGz5urq6hiojTqETkxrikrvI+s91u5/V5cZiCCjffqD6/33q8tvPGPk3TpogExpg8y1/uIObb1VB9f7/1dW3XjX0URc0kSb5PJpNGo9F4eL/2agexZ7j1RvX6/dbXtV039qfT6c1a25lOp/F4PN49svZFD2Le3XWY8vX91te1i9rYd7vdXH93mqaBsG0oVK1P/WUoe9tQ5r7eJ0ItmOu2YbFYjLbb7dAYc8hxedvv93+EYdh9dKHvgb0rQvUkz7bB9W2cJIkVkffZbPYxHA4fCt33wN4VoVaIy9tY5F9s1tqkrH29L4RaQXkPcVWPzQVfpqACoUIFQoUKXveojt/LKz3Xw3N5C7WIUUuV53p4Lm+hFjVqqepcD8/lfTzFqAVF4DAFFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVCBUqECpUIFSoQKhQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUKFu/7P1H6/t4/+4uPxaEVEoiiym82m88z1Wq+t9b5d19/T11/LXqGUie0GBwAAAABJRU5ErkJggg=="">",0(0.0%)
4,density[float64],Mean (sd) : 1.0 (0.0)min < med < max:1.0 < 1.0 < 1.0IQR (CV) : 0.0 (232.0),151 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAA

## 3. Exploring the data

In [6]:
# define numerical & categorical columns
numeric_features = [feature for feature in data.columns if data[feature].dtype != 'O']
categorical_features = [feature for feature in data.columns if data[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numeric_features), numeric_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

We have 4 numerical features : ['fixed_acidity', 'residual_sugar', 'alcohol', 'density']

We have 1 categorical features : ['quality_label']


In [8]:
# proportion of count data on categorical columns
print(data['quality_label'].value_counts(normalize=True) * 100)

medium    35.5
high      34.3
low       30.2
Name: quality_label, dtype: float64


#### Insights
- Our quality label is faily balanced in the dataset

In [17]:
import plotly.io as pio
pio.renderers.default = 'iframe'

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy.stats import gaussian_kde

# Setup subplot grid: 4 rows x 3 columns
rows = 4
cols = 3
fig = make_subplots(rows=rows, cols=cols, subplot_titles=numeric_features)

# Loop through each numerical feature
for idx, feature in enumerate(numeric_features):
    row = idx // cols + 1
    col = idx % cols + 1
    
    # Compute KDE using scipy (as Plotly doesn't have built-in KDE)
    x = data[feature].dropna()
    kde = gaussian_kde(x)
    x_range = np.linspace(x.min(), x.max(), 200)
    y_kde = kde(x_range)

    # Add KDE line to subplot
    fig.add_trace(
        go.Scatter(x=x_range, y=y_kde, mode='lines', name=feature, line=dict(color='blue')),
        row=row, col=col
    )

# Layout settings
fig.update_layout(
    height=400, width=600,
    title_text="Univariate Analysis of Numerical Features",
    title_font=dict(size=20, family="Arial", color="black"),
    showlegend=False
)

fig.show()